In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
import json
from kafka import KafkaProducer
from kafka.admin import KafkaAdminClient, NewTopic
from kafka.errors import KafkaError
import random
import os
import logging
import signal
import pandas as pd

# --- Cấu hình Logging ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# --- Cấu hình ---
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--log-level=3')

URL_TARGET = 'https://www.thegioididong.com/laptop'

KAFKA_BROKER_URL = os.getenv('KAFKA_BROKER_URL', 'localhost:9092')
KAFKA_TOPIC = 'device-data'

# --- Khởi tạo WebDriver ---
driver = None
producer = None

def signal_handler(sig, frame):
    logger.info("Received interrupt signal. Cleaning up...")
    if producer:
        producer.flush(timeout=60)
        producer.close()
    if driver:
        driver.quit()
    logger.info("Cleanup completed. Exiting.")
    exit(0)

signal.signal(signal.SIGINT, signal_handler)
signal.signal(signal.SIGTERM, signal_handler)

try:
    logger.info("Initializing WebDriver...")
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)
    driver.implicitly_wait(5)
    logger.info("WebDriver initialized.")
except Exception as e:
    logger.error(f"CRITICAL: Error initializing WebDriver: {e}")
    exit()

# --- Kiểm tra và tạo Kafka Topic ---
def ensure_kafka_topic():
    try:
        admin_client = KafkaAdminClient(bootstrap_servers=[KAFKA_BROKER_URL])
        topic_list = admin_client.list_topics()
        if KAFKA_TOPIC not in topic_list:
            logger.info(f"Topic {KAFKA_TOPIC} does not exist. Creating...")
            new_topic = NewTopic(name=KAFKA_TOPIC, num_partitions=1, replication_factor=1)
            admin_client.create_topics(new_topics=[new_topic], validate_only=False)
            logger.info(f"Topic {KAFKA_TOPIC} created successfully.")
        else:
            logger.info(f"Topic {KAFKA_TOPIC} already exists.")
        admin_client.close()
    except Exception as e:
        logger.error(f"ERROR ensuring Kafka topic: {e}")
        raise

ensure_kafka_topic()

# --- Khởi tạo Kafka Producer ---
try:
    logger.info(f"Connecting to Kafka broker at {KAFKA_BROKER_URL}...")
    producer = KafkaProducer(
        bootstrap_servers=[KAFKA_BROKER_URL],
        value_serializer=lambda v: json.dumps(v, ensure_ascii=False).encode('utf-8'),
        retries=5,
        acks='all'
    )
    logger.info("Successfully connected to Kafka.")
except Exception as e:
    logger.error(f"CRITICAL: Error connecting to Kafka: {e}")
    if driver:
        driver.quit()
    exit()

# --- Hàm gửi dữ liệu vào Kafka ---
def on_send_success(record_metadata):
    pass

def on_send_error(excp):
    logger.error(f"ERROR sending message to Kafka: {excp}")

def send_to_kafka_async(producer, topic, data, max_retries=3):
    if not data or not producer:
        return False
    for attempt in range(max_retries):
        try:
            producer.send(topic, value=data).add_callback(on_send_success).add_errback(on_send_error)
            return True
        except Exception as e:
            logger.error(f"Attempt {attempt+1}/{max_retries} failed: {e}")
            if attempt < max_retries - 1:
                time.sleep(2 ** attempt)  # Exponential backoff
            else:
                logger.error(f"Failed to send after {max_retries} attempts.")
                return False

# --- Vòng lặp Crawl chính ---
logger.info(f"Starting crawl from URL: {URL_TARGET}")
successful_sends_attempted = 0
failed_prepares = 0

try:
    logger.info(f"Loading page: {URL_TARGET}")
    driver.get(URL_TARGET)

    # Chờ danh sách sản phẩm xuất hiện
    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.CLASS_NAME, 'listproduct'))
    )

    # Nhấn nút "Xem thêm" cho đến khi không còn nút này
    logger.info("Nhấn nút 'Xem thêm' để tải hết sản phẩm...")
    max_attempts = 3  # Số lần thử lại tối đa nếu gặp lỗi
    attempts = 0

    while True:
        try:
            # Lưu số lượng sản phẩm hiện tại
            product_list = driver.find_element(By.CLASS_NAME, 'listproduct')
            current_product_count = len(product_list.find_elements(By.TAG_NAME, 'li'))

            # Kiểm tra xem nút "Xem thêm" có tồn tại không
            view_more_buttons = driver.find_elements(By.CLASS_NAME, 'see-more-btn')
            if not view_more_buttons:
                logger.info("Không tìm thấy nút 'Xem thêm', đã tải hết sản phẩm.")
                break

            # Lấy nút "Xem thêm" đầu tiên
            view_more_button = WebDriverWait(driver, 30).until(
                EC.element_to_be_clickable((By.CLASS_NAME, 'see-more-btn'))
            )

            # Cuộn đến nút "Xem thêm"
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", view_more_button)
            time.sleep(1)  # Đợi cuộn hoàn tất

            # Nhấn nút
            view_more_button.click()
            logger.info("Đã nhấn nút 'Xem thêm', chờ tải thêm sản phẩm...")

            # Chờ danh sách sản phẩm cập nhật
            WebDriverWait(driver, 30).until(
                lambda d: len(d.find_element(By.CLASS_NAME, 'listproduct').find_elements(By.TAG_NAME, 'li')) > current_product_count
            )
            logger.info(f"Số sản phẩm hiện tại: {len(driver.find_element(By.CLASS_NAME, 'listproduct').find_elements(By.TAG_NAME, 'li'))}")
            time.sleep(2)  # Đợi dữ liệu ổn định

            # Đặt lại số lần thử nếu nhấn thành công
            attempts = 0

        except Exception as e:
            # Kiểm tra xem nút "Xem thêm" có còn tồn tại không
            view_more_buttons = driver.find_elements(By.CLASS_NAME, 'see-more-btn')
            if not view_more_buttons:
                logger.info("Xác nhận: Không còn nút 'Xem thêm', dừng lại.")
                break

            # Nếu nút vẫn còn, tăng số lần thử
            attempts += 1
            logger.error(f"Lỗi khi nhấn nút 'Xem thêm' (lần {attempts}/{max_attempts}): {str(e)}")
            
            if attempts >= max_attempts:
                logger.info(f"Đạt số lần thử tối đa ({max_attempts}), dừng lại.")
                break

            # Đợi lâu hơn và thử lại
            time.sleep(5)
            continue

    # Lấy nội dung HTML sau khi tải hết sản phẩm
    html_content = driver.page_source
    soup = BeautifulSoup(html_content, "html.parser")

    # Tìm danh sách sản phẩm
    product_list = soup.find('ul', class_='listproduct')
    if not product_list:
        logger.error("Không tìm thấy danh sách sản phẩm!")
        driver.quit()
        exit()

    # Lặp qua từng sản phẩm
    products = product_list.find_all('li', class_='item')
    logger.info(f"Tìm thấy {len(products)} sản phẩm.")

    # Danh sách lưu trữ dữ liệu sản phẩm
    all_products_data = []

    for index, product in enumerate(products):
        product_data = {}

        # Lấy thẻ <div class="main-contain"> chứa thông tin chính
        main_contain = product.find('a', class_='main-contain')
        if not main_contain:
            logger.warning(f"Bỏ qua sản phẩm {index + 1}: Không tìm thấy thẻ main-contain")
            continue

        # 1. Tên sản phẩm (trong thẻ <h3>)
        name_tag = main_contain.find('h3')
        if name_tag:
            new_model_tag = name_tag.find('span', class_='newModel')
            if new_model_tag:
                new_model_tag.decompose()
            product_data['name'] = name_tag.text.strip() if name_tag else "N/A"
        else:
            product_data['name'] = "N/A"
        
        RAM_ROM = main_contain.find('div', class_='item-compare')
        if RAM_ROM:
            RAM_ROM = RAM_ROM.find_all('span')
            if len(RAM_ROM) >= 2:
                product_data['RAM'] = RAM_ROM[0].text.strip() if RAM_ROM[0] else "N/A"
                product_data['ROM'] = RAM_ROM[1].text.strip() if RAM_ROM[1] else "N/A"
            else:
                product_data['RAM'] = "N/A"
                product_data['ROM'] = "N/A"

        data_brand = main_contain.get('data-brand')
        if data_brand:
            product_data['brand'] = data_brand.strip()
        else:
            product_data['brand'] = "N/A"
        
        color = main_contain.get('data-color')
        if color:
            product_data['color'] = color.strip()
        else:
            product_data['color'] = "N/A"

        data_cate = main_contain.get('data-cate')
        if data_cate:
            product_data['category'] = data_cate.strip()
        else:
            product_data['category'] = "N/A"

        data_price = main_contain.get('data-price')
        if data_price:
            product_data['price'] = data_price.strip()
        else:
            product_data['price'] = "N/A"
        
        price_old = main_contain.get('price-old')
        if price_old:
            product_data['price_old'] = price_old.strip()
        else:
            product_data['price_old'] = data_price
        
        percent = (float(product_data['price_old']) - float(product_data['price'])) / float(product_data['price_old']) * 100 if float(product_data['price_old']) > 0 else 0
        product_data['percent'] = str(int(percent))

        # 4. Thông tin đánh giá (trong thẻ <div class="rating_Compare has_compare has_quantity">)
        rating_div = product.find('div', class_='rating_Compare has_compare has_quantity')
        if rating_div:
            num_star = rating_div.find('b')
            if num_star:
                numstar = num_star.text.strip()
                if numstar == 'Chưa có đánh giá':
                    numstar = "0"
                product_data['rating'] = numstar
            else:
                product_data['rating'] = "0"

            toltal_sell = rating_div.find('span')
            if toltal_sell:
                totalcell = toltal_sell.text.split()
                n = len(totalcell)
                totalCell = totalcell[n - 1]
                # Chuyển đổi số lượng bán nếu có "K"
                if 'k' in totalCell:
                    number = totalCell.replace('k', '').replace(',', '.')
                    totalCell = str(int(float(number) * 1000))
                product_data['sold'] = totalCell
            else:
                product_data['sold'] = "0"
        else:
            product_data['rating'] = "Chưa có đánh giá"
            product_data['sold'] = "N/A"
            if product_data['rating'] == 'Chưa có đánh giá':
                product_data['rating'] = "0"
            if product_data['sold'] == 'N/A':
                product_data['sold'] = "0"

        # Thêm dữ liệu sản phẩm vào danh sách
        all_products_data.append(product_data)
        logger.info(f"Đã crawl sản phẩm {index + 1}: {product_data['name']}")

        # Gửi dữ liệu vào Kafka
        kafka_data = {
            'crawl_timestamp': int(time.time() * 1000),
            'product_id': index + 1,
            'name': product_data['name'],
            'brand': product_data['brand'],
            'category': product_data['category'],
            'color': product_data['color'],
            'price': product_data['price'],
            'price_old': product_data['price_old'],
            'RAM': product_data['RAM'],
            'ROM': product_data['ROM'],
            'percent': product_data['percent'],
            'rating': product_data['rating'],
            'sold': product_data['sold'],
            'source': URL_TARGET
        }
        if send_to_kafka_async(producer, KAFKA_TOPIC, kafka_data):
            successful_sends_attempted += 1
            logger.info(f"Successfully sent data for product {product_data['name']} to Kafka")
        else:
            failed_prepares += 1
            logger.error(f"Failed to send data for product {product_data['name']} to Kafka")

finally:
    logger.info("-" * 30)
    logger.info("Crawl loop finished.")
    logger.info(f"Total Kafka send attempts prepared: {successful_sends_attempted}")
    logger.info(f"Total Kafka send preparation failures: {failed_prepares}")
    if producer:
        logger.info("Flushing Kafka producer (waiting for pending messages)...")
        try:
            producer.flush(timeout=60)
            logger.info("Kafka producer flushed.")
        except Exception as flush_e:
            logger.error(f"ERROR during producer flush: {flush_e}")
        finally:
            logger.info("Closing Kafka producer.")
            producer.close()
    if driver:
        logger.info("Closing WebDriver.")
        driver.quit()
    logger.info("Script finished.")

    # In dữ liệu và lưu vào file CSV (tùy chọn)
    if all_products_data:
        logger.info("\n---- Dữ liệu sản phẩm ----")
        for data in all_products_data:
            logger.info(data)
        logger.info("\n---- Kết thúc ----")

        # Lưu dữ liệu vào file CSV với thứ tự cột được chỉ định
        df = pd.DataFrame(all_products_data)
        df = df[['name', 'brand', 'category', 'color', 'price', 'price_old', 'RAM', 'ROM', 'percent', 'rating', 'sold']]
        df.to_csv('laptop_data.csv', index=False, encoding='utf-8-sig')
        logger.info("\nDữ liệu đã được lưu vào file 'laptop_data.csv'")
    else:
        logger.info("Không có dữ liệu để lưu!")

2025-05-18 15:30:38,289 - INFO - Initializing WebDriver...
2025-05-18 15:30:38,295 - INFO - ====== WebDriver manager ======
2025-05-18 15:30:45,849 - INFO - Get LATEST chromedriver version for google-chrome
2025-05-18 15:30:46,191 - INFO - Get LATEST chromedriver version for google-chrome
2025-05-18 15:30:46,427 - INFO - Driver [C:\Users\ADMIN\.wdm\drivers\chromedriver\win64\136.0.7103.94\chromedriver-win32/chromedriver.exe] found in cache
2025-05-18 15:30:50,244 - INFO - WebDriver initialized.
2025-05-18 15:30:50,244 - INFO - <BrokerConnection client_id=kafka-python-2.1.5, node_id=bootstrap-0 host=localhost:9092 <connecting> [IPv6 ('::1', 9092, 0, 0)]>: connecting to localhost:9092 [('::1', 9092, 0, 0) IPv6]
2025-05-18 15:30:50,630 - INFO - <BrokerConnection client_id=kafka-python-2.1.5, node_id=bootstrap-0 host=localhost:9092 <checking_api_versions_recv> [IPv6 ('::1', 9092, 0, 0)]>: Broker version identified as 2.6
2025-05-18 15:30:50,631 - INFO - <BrokerConnection client_id=kafka-py